In [1]:
import DeepLabModel
import os, urllib
from PIL import Image
from io import BytesIO
import numpy as np
from skimage.io import imshow
import matplotlib.pyplot as plt
from matplotlib import gridspec
from ipywidgets import FloatSlider, interact, fixed, HBox, VBox, Label, Button, Output, IntProgress
%pdb 0
plt.rcParams['figure.max_open_warning'] = False
from multiprocessing import Process, Queue
import pandas as pd
import copy

Automatic pdb calling has been turned OFF


### Multiagent Negotiation applied to image segmentators


In [2]:
def show_proposals(task, proposals, utilities):
        plt.figure(figsize=(16,9), dpi=300)
        # Finding all the labels that appear in the proposals for the legend
        unique_labels = np.array([], dtype=np.int)
        for p in proposals:
            unique_labels = np.union1d(unique_labels, np.unique(p))
        unique_labels = unique_labels[unique_labels!=255] # We ignore the void label (see VOC2012 docs)
                
        max_cols = len(proposals)+2

        grid_spec = gridspec.GridSpec(1, max_cols, width_ratios= [6] + [6 for i in proposals]+ [2])
        # Show input
        plt.subplot(grid_spec[0])
        plt.imshow(task)
        plt.axis('off')
        plt.title('Input')

        for p, prop in enumerate(proposals):
            plt.subplot(grid_spec[p+1])
            plt.imshow(DeepLabModel.label_to_color_image(prop).astype(np.uint8))
            plt.axis('off')
            plt.title('Model {}'.format(p+1))

        ax = plt.subplot(grid_spec[max_cols-1])
        plt.imshow(
            FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest', aspect=0.3)
        ax.yaxis.tick_right()
        plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
        plt.xticks([], [])
        ax.tick_params(width=0.0)
        plt.grid(False)
        plt.show()

def show_agreement(agreement, i):
        labels = agreement.keys()
        plt.figure(figsize=(16,9), dpi=300)
        for lpos, lab in enumerate(labels):
            agr = agreement[lab]
            plt.subplot(1, len(labels), lpos+1)
            plt.title("Agreement {} for label {}".format(i, LABEL_NAMES[lab]))
            plt.set_cmap('gray')
            plt.axis('off')
            plt.imshow(agr)
            
def show_stats(log):
    plt.figure(figsize=(16,9))
    plt.subplot(1, 2, 1)
    util = np.array(log['util']).transpose()
    plt.title("Utility")
    for a, agent_util in enumerate(util):
        plt.plot(agent_util, label="Agent {}".format(a+1))
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.title("Remaining Pixels %")
    progress = np.array(log['remaining_points'])
    progress = progress / progress[0]
    plt.plot(progress)
    

def get_remaining_points(agreement):
    stacked = np.stack([a for k, a in agreement.items()])
    return np.count_nonzero(np.logical_and(stacked!=0, stacked!=1))
    

In [3]:
class Agent():
    def __init__(self, agentname, model_name, alpha, seed=None):
        self.agentname=agentname
        self.model_name = model_name
        self.task = None
        self.optimal = None
        self.alpha = alpha
        self.seed = seed
        self.model = None
        
    def load_model(self):
        _DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
        model_dir = '../models/'
        _MODEL_URLS = {
            'mobilenetv2_coco_voctrainaug':
                'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
            'mobilenetv2_coco_voctrainval':
                'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
            'xception_coco_voctrainaug':
                'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
            'xception_coco_voctrainval':
                'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
        }
        
        tarname = _MODEL_URLS[self.model_name]
        
        download_path = os.path.join(model_dir, tarname)
        if not os.path.isfile(download_path):
            print('downloading {}, this might take a while...'.format(modelname))
            urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[modelname],
                               download_path)
            print('download completed!')

        else:
            print("model already present at {}".format(download_path))

        print("Agent {} is loading...".format(self.agentname))
        self.model = DeepLabModel.DeepLabModel(download_path)
        print("Agent {} is ready.".format(self.agentname))
        
    
    def run(self, inqueue, outqueue):
        self.inqueue = inqueue
        self.outqueue = outqueue
        self.load_model()
        # The first element we expect is the task
        print("Agent {} is waiting for a task".format(self.agentname))
        image = self.inqueue.get()
        print("Agent {} got a task: {}".format(self.agentname, image.size))
        optimal, utility = self.new_task(image)
        # Asnwer back
        self.outqueue.put((self.optimal, self.utility(self.optimal)))
        # Now the agents only answer to agreements until a None is received
        received = self.inqueue.get()
        while received is not None:
            proposal, utility = self.propose()
            self.outqueue.put((proposal, utility))
            received = self.inqueue.get()
        print("Agent {} is stopping".format(self.agentname))
        
        
    def new_task(self, image):
        self.task, self.optimal = self.model.run(image)
        return optimal, utility
    
    def _dice_score(x, y):
        x_and_y = np.count_nonzero(np.logical_and(x, y))
        card_x = np.count_nonzero(x)
        card_y = np.count_nonzero(y)
        return 2*x_and_y/(card_x+card_y+1e-16)
    
    def utility(self, proposal):
        # Since we are using the smooth dice loss, we have to split each label in the segmentation and sum up their scores
        dist =lambda x,y: 1.-Agent._dice_score(x,y)
        labels = np.unique(proposal)
        util = np.sum([dist(proposal==l, self.optimal==l) for l in labels])
        return util
    
    def _build_proposal(self, agreement):
        
        negotiation_map = np.zeros_like(agreement[0]) # For keeping track on what pixels are still to negotiate
        proposal = np.zeros_like(agreement[0]) # Initialize an empty proposal
        for l, conf in agreement.items(): # For each "probability map" received in the agreement
            proposal[conf==1] = l
            negotiation_map[conf==1] = 1
        
        x, y=np.where(negotiation_map!=1)
        for x, y in zip(x, y):
            # For each pixel, pick among the possible labels with the relative probability
            labels = list(agreement.keys())
            probs = [agreement[l][x,y] for l in labels]
            belief = self.optimal[x,y] # The agent thinks that the right label is this one
            if np.random.random() < self.alpha[belief]:
                proposal[x,y] = belief
            else:
                proposal[x,y] = np.random.choice(labels, p=probs)
        proposal = proposal.astype(np.int)
        utility = self.utility(proposal)
        return proposal, utility
    
    def propose(self):
        while True:
            print("Agent {} waiting for agreement".format(self.agentname))
            agreement = self.inqueue.get() # This is blocking until a message arrives
            if agreement is None:
                print("Agent {} exiting negotiation".format(self.agentname))
                break
            np.random.seed(self.seed)
            proposal, utility = self._build_proposal(agreement)
            # Send proposal to the mediator
            self.outqueue.put((proposal, utility))
            print("Agent {} sent a proposal".format(self.agentname))
        

In [4]:
class Mediator():
    def __init__(self, agents, timeout=10):
        self.agents = agents # Reference to agents
        # Message queues to communicate with the agents
        self.output_queues = [Queue() for a in agents]
        self.input_queues =  [Queue() for a in agents]
        self.agentpool = [] # Processes simulating the agents
        self.timeout = timeout # Iteration steps for each negotiation() call
        self.last_step=0
        
        self.agentpool = [Process(target=a.run, 
                                  args=(self.output_queues[a_id], self.input_queues[a_id])) for a_id, a in enumerate(self.agents)]
        for a in self.agentpool:
            a.start()
        print("Agents spawned")
        
    def start_new_task(self, image):
        self.task = image
        # Starting the new processes
        for q in self.output_queues:
            q.put(image)
            print("Task submitted to agent")
            
        # getting (proposal, utility) from each agent
        received_proposals = [q.get() for q in self.input_queues]
        print("Got proposals")
        
        # Packing and storing the received initial proposals
        self.initial_proposals = np.array([a[0] for a in received_proposals]) # Shape (agents, h, w)
        utilities_array = np.array([a[1] for a in received_proposals]) # Shape (agents, )
        self.initial_utilities = np.transpose(np.tile(utilities_array, [self.initial_proposals.shape[1], self.initial_proposals.shape[2], 1]), [2,0,1]) # Shape (agents, h, w)
        self.last_proposals = self.initial_proposals
        self.last_utilities = self.initial_utilities
        
        show_proposals(self.last_proposals, self.last_utilities)   
    
    def negotiation(self, show_every=100):
        for i in range(self.last_step, self.last_step+self.timeout):
            self.last_step = i
            # Agreement must be built label-wise (averaging over the values could produce new labels)
            
            all_proposed_labels = np.unique(self.last_proposals)
            agreements = {l:None for l in all_proposed_labels}
            for l in all_proposed_labels:
                prop = (self.last_proposals==l)
                util = (self.last_utilities==l)
                agreement_l = np.sum(prop*(1+util), axis=0)/np.sum(1+util, axis=0)
                agreements[l] = agreement_l
            
            
            # Propose the new agreement to the agents
            received_proposals = [agent.propose(agreements) for agent in self.agents]
            
            # Publish an agreement
            [q.put(agreements) for q in self.output_queues]
            
            # getting (proposal, utility) from each agent
            received_proposals = [q.get() for q in self.input_queues]
        
            self.last_proposals = np.array([a[0] for a in received_proposals]) # Shape (agents, h, w)
            utilities_array = np.array([a[1] for a in received_proposals]) # Shape (agents, )
            self.last_utilities = np.transpose(np.tile(utilities_array, [self.last_proposals.shape[1], self.last_proposals.shape[2], 1]), [2,0,1]) # Shape (agents, h, w)
        
            if i % show_every == 0:
                show_agreement(agreements, i)
                show_proposals(self.last_proposals, self.last_utilities)
        return agreements

#### Defining label colors and names

In [5]:
LABEL_NAMES = np.asarray(
    [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])
FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = DeepLabModel.label_to_color_image(FULL_LABEL_MAP)

## Initializing Agents and Mediator

### Defining agent parameters

In [6]:
loaded = pd.read_csv('params.csv', index_col=0)

In [7]:
AGENT_MODELS = {
            'mobile_aug':
                'mobilenetv2_coco_voctrainaug',
            'mobile_val':
                'mobilenetv2_coco_voctrainval',
            'xception_aug':
                'xception_coco_voctrainaug',
            'xception_val':
                'xception_coco_voctrainval',
        }

alphasliders = {m:[FloatSlider(description=l, min=0., max=1., step=0.01, value=loaded.loc[m][l]) for l in LABEL_NAMES] for a, m in AGENT_MODELS.items()}
sliders = [VBox([Label(model)]+alphasliders[model]) for model in alphasliders.keys()]
HBox(sliders)

## Demo on a random Image

In [8]:
# Download an image and start negotiation
url = 'http://farm8.static.flickr.com/7920/47425338881_de208df2ff_m.jpg'
#url = 'https://cdn.vox-cdn.com/uploads/chorus_asset/file/8327269/Screen_Shot_2017_04_12_at_3.54.13_PM.png'
#url = 'https://proxy.duckduckgo.com/iu/?u=https%3A%2F%2Fimages.fastcompany.net%2Fimage%2Fupload%2Fw_596%2Cc_limit%2Cq_auto%3Abest%2Cf_auto%2Cfl_lossy%2Ffc%2F1683481-inline-s-2-dog-tv-share-the-remote-control.jpg&f=1'
#url = 'https://www.zastavki.com/pictures/originals/2013/People_Shadows_of_people_looked_047805_.jpg'
f = urllib.request.urlopen(url)
jpeg_str = f.read()
original_im = Image.open(BytesIO(jpeg_str))

alpha = {m: [slider.value for slider in alphasliders[m]] for m in alphasliders.keys()}
agents = [Agent(agentname, modelname, alpha[modelname]) for agentname, modelname in AGENT_MODELS.items()]
mediator = Mediator(agents)
print("Mediator Running")


model already present at ../models/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz
Agent mobile_val is loading...
model already present at ../models/deeplabv3_pascal_trainval_2018_01_04.tar.gz
Agents spawned
Mediator Running
Agent xception_val is loading...
model already present at ../models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
Agent mobile_aug is loading...
model already present at ../models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz
Agent xception_aug is loading...
Agent mobile_aug is ready.
Agent mobile_aug is waiting for a task
Agent mobile_aug got a task: (240, 160)
Agent mobile_val is ready.
Agent mobile_val is waiting for a task
Agent mobile_val got a task: (240, 160)


Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-0c2dc8a55e33>", line 45, in run
    self.load_model()
  File "<ipython-input-3-0c2dc8a55e33>", line 38, in load_model
    self.model = DeepLabModel.DeepLabModel(download_path)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1319, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/DCSeg/src/DeepLabModel.py", line 40, in __init__
    self.sess = tf.Session(graph=self.graph)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client

In [9]:
log = {'util':[], 'remaining_points':list()}
init_proposal, init_utilities = mediator.start_new_task(original_im)  
next_step = enumerate(mediator.negotiation(timeout=10000))
display(Label("Current Step"))
step_progress = IntProgress(max=1)
display(step_progress)
display(Label("Total Progress"))
pixel_progress = FloatProgress(max=1.0)
display(pixel_progress)
grid_spec = None
from ipywidgets import IntText

def run_negotiation(n):
    if n<len(log['remaining_points']):
        return
    step_progress.max = n
    for i, (agreement, proposals, utilities) in next_step:
        step_progress.value = i
        current_rp = get_remaining_points(agreement)
        log['util'].append(utilities.mean(axis=2).mean(axis=1))
        log['remaining_points'].append(current_rp)
        pixel_progress.value = 1 - current_rp/log['remaining_points'][0]
        #print("Done {} of {}, ({})%".format(current_rp, log['remaining_points'][0], pixel_progress.value))
        if i == n-1 or pixel_progress.value >= 0.99:
            show_proposals(original_im, init_proposal, init_utilities)
            show_agreement(agreement, i)
            show_proposals(original_im, proposals, utilities)
            show_stats(log)
            return

interact(run_negotiation, n=IntText(value=1));

Task submitted to agent
Task submitted to agent
Task submitted to agent
Task submitted to agent


KeyboardInterrupt: 

In [ ]:
# TODO: Prova giusto la versione senza gpu. Se non va nemmeno quella droppa l'idea e ripristina il backup